In [1]:
# загружаем основные библиотеки
import numpy as np
import pandas as pd

In [2]:
# пишем функцию, которая вычисляет 
# формулу гаверсинусов
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [3]:
# загружаем данные, на которых будем тестировать 
# нашу функцию
df = pd.read_csv('C:/data/new_york_hotels.csv', encoding='cp1252')
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


# Базовое итерирование

In [4]:
# задаем функцию, которая просто перебирает все строки 
# и возвращает серию с расстояниями, вычисленными
# по формуле гаверсинусов
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [5]:
%%timeit

# запускаем итеративную функцию haversine
df['distance'] = haversine_looping(df)

696 ms ± 53.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Итерирование с помощью метода .iterrows()

In [6]:
%%timeit

# запускаем итерирование с помощью
# метода .iterrrows()
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

215 ms ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Более лучший способ итерирования с помощью метода .apply()

In [7]:
%%timeit

# применяем функцию haversine с помощью
# метода .apply()
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'],
                                                row['longitude']), axis=1)

81.2 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Векторизация c помощью объектов Series

In [8]:
%%timeit 

# векторизированная реализация функции haversine, 
# используем объекты Series целиком
df['distance'] = haversine(40.671, -73.985,df['latitude'], df['longitude'])

2.09 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Векторизация с помощью массивов NumPy

In [9]:
%%timeit

# векторизированная реализация функции haversine, 
# используем вместо объектов Series массивы NumPy
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

372 µs ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
